# Deutsch's algorithm
## Quick overview
The algorithm allows us to determine whether the function is constant (all output values are the same) or balanced (2 output values with exactly 50 / 50 distribution). A classical computer needs $2^{n - 1} + 1$ function calls in the worst case to determine if the function is balanced. QC can do that in just one function call.  \
\
The circuit for the function with only 1 input would look like this: \
![deutsch circuit](https://drive.google.com/uc?id=1fDbUvoYZGhxN9FuJYFInof-Ig-iANCm8) \

If a function accepts more then one parameter - the number of qubits will be equivalent to the number of inputs. In that case, the algorithm is called Deutsch–Jozsa and looks like this:
![deutsch circuit](https://drive.google.com/uc?id=13SgIllh_UtUSkJy6YTa6FId1nth4WRD9) 

### All possible cases for single input function.

Let's describe all possible functions that can exist with a single bit input. \
There are only 4 possible variations. \

The functions    | Type         | Matrix version where $x = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $\begin{pmatrix} 0 \\ 1 \end{pmatrix}$
-----------      | ------------ | ---
$f(x) = 0$       | Constant     | $\begin{pmatrix} 1 & 1 \\ 0 & 0 \end{pmatrix}$
$f(x) = 1$       | Constant     | $\begin{pmatrix} 0 & 0 \\ 1 & 1 \end{pmatrix}$
$f(x) = \neg x$  | negation     | $\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$
$f(x) = x$       | identity     | $\begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$

One of the rules of QC says that quantum functions have to be reversible, which means the matrix has to be unitary ($A^TA = I$) . \
To avoid creating matrixes that represent these functions we can simply build the black box $U_f$ with existing gates form our library. So $U_f$ can be considered completely reversible \

--- 
### Examples. 
Let's implement a circuit with $f(x) = 1$. \
The first qubit will represent our input and the last one will represent our output.

In [ ]:
#instaling the library \
!git clone https://github.com/katolikyan/QCSimulator.git
!pip3 install QCSimulator/

### $f(x) = 1$

In [ ]:
import qcsimulator as qcs

# creating a circuit with 2 qubits on it.
circuit = qcs.circuit_init(2)

circuit.x(-1) # applying not gate to the last qubit
circuit.h(0)  # Hadamard on qubit 0
circuit.h(1)  # Hadamard on qubit 1

# We don't have to do anything in this case because our constant function
# representation will look like 2 wires without any gates on it.
# -----
# -----

circuit.h(0) # Hadamard on qubit 0 again.
result = circuit.execute() # execution of the circuit.

# printing specific qubit collapse probability. qubit measurement equivalent.
print("qubit 0 collapsing probability: \n", \
      result.get_single_qubit_probability(0), "\n")

# We can also doublecheck that the first qubit will always collapse into 1 or 0
# by looking at all probabilities. 
print(result.get_all_probabilities())

qubit 0 collapsing probability: 
 {'0': 0.9999999999999996, '1': 5.004680467665245e-34} 

{'00': 0.4999999999999998, '01': 2.5023402338326227e-34, '10': 0.4999999999999998, '11': 2.5023402338326227e-34}


We care only about measuring inputs. The Deutsch algorithm is deterministic so we need to only know if the first bit always collapsing to `0` (which means the function is constant) or to 1 (which means it is balanced). \
In the case above we can notice that the first qubit will always collapse to `0`. \
Let's recreate the rest of the functions. 

### $f(x) = 0$

In [ ]:
circuit = qcs.circuit_init(2)

circuit.x(-1)
circuit.h(0)
circuit.h(1)

# Uf will look like:
# -----
# --X--
circuit.x(1)

circuit.h(0)
result = circuit.execute()
print("qubit 0 collapsing probability: \n", \
      result.get_single_qubit_probability(0), "\n")
print(result.get_all_probabilities())

qubit 0 collapsing probability: 
 {'0': 0.9999999999999996, '1': 0.0} 

{'00': 0.4999999999999998, '01': 0.0, '10': 0.4999999999999998, '11': 0.0}


### $f(x) = \neg x$

In [ ]:
circuit = qcs.circuit_init(2)

circuit.x(-1)
circuit.h(0)
circuit.h(1)

# Uf will look like:
# --*--
#   |
# --X--
circuit.cx(0, 1)

circuit.h(0)
result = circuit.execute()
print("qubit 0 collapsing probability: \n", \
      result.get_single_qubit_probability(0), "\n")
print(result.get_all_probabilities())

qubit 0 collapsing probability: 
 {'0': 1.0573994819069698e-33, '1': 0.9999999999999996} 

{'00': 5.286997409534849e-34, '01': 0.4999999999999998, '10': 5.286997409534849e-34, '11': 0.4999999999999998}


### $f(x) = x$

In [ ]:
circuit = qcs.circuit_init(2)

circuit.x(-1)
circuit.h(0)
circuit.h(1)

# Uf will look like:
# -----*--
#      |
# --X--X--
circuit.x(0)
circuit.cx(0, 1)

circuit.h(0)
result = circuit.execute()
print("qubit 0 collapsing probability: \n", \
      result.get_single_qubit_probability(0), "\n")
print(result.get_all_probabilities())

qubit 0 collapsing probability: 
 {'0': 0.0, '1': 0.9999999999999996} 

{'00': 0.0, '01': 0.4999999999999998, '10': 0.0, '11': 0.4999999999999998}


### Multiple argument function. Deutsch–Jozsa.
Just to satisfy our curiosity let's try to implement a circuit with a function like $f(x_1, x_2)$ \
Let's make a balanced one, because for constant function $U_f$ would not be too different from something we have implemented for a single argument algorithm.

Let's say that our function maps inputs as follows: \
`00: 1, 01: 0, 10: 0, 11: 1`
Thus we have balanced function and we expect `qubit 0` and `qubit 1` to collapse only in `1`. \
Let's create a circuit for it. 

In [ ]:
n_qubits = 3
circuit = qcs.circuit_init(n_qubits) # We need 3 qubits now.

circuit.x(-1)                        # X on last one
for i in range(n_qubits):            # H on every one
  circuit.h(i)

# Uf will look like:
# 0 --*-----
#     |  
# 1 --|--*--
#     |  |
# 2 --X--X--
circuit.cx(0, 2)
circuit.cx(1, 2)

for i in range(n_qubits - 1):        # H on every one but last one
  circuit.h(i)
  
result = circuit.execute()

# Measuring 2 single qubits. expecting prob of both to be 1.
print("qubit 0 collapsing probability: \n", \
      result.get_single_qubit_probability(0))
print("qubit 1 collapsing probability: \n", \
      result.get_single_qubit_probability(1), "\n")
print(result.get_all_probabilities())

qubit 0 collapsing probability: 
 {'0': 6.07716335728627e-64, '1': 0.9999999999999989}
qubit 1 collapsing probability: 
 {'0': 1.8142106940266839e-34, '1': 0.9999999999999989} 

{'000': 0.0, '001': 9.071053470133419e-35, '010': 3.038581678643135e-64, '011': 0.49999999999999944, '100': 0.0, '101': 9.071053470133419e-35, '110': 3.038581678643135e-64, '111': 0.49999999999999944}


--- 
#### Thank you!
All info and updates for the lib can be found on the project's repo on GitHub: \
https://github.com/katolikyan/QCSimulator \
Do not hesitate to open an issue and contribute :)
